sdi2100007 - Μιχαήλ Αρετάκης <br>
sdi2100222 - Γεώργιος Παπαϊωάννου

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

Στο παραπάνω κελί, κάνουμε mount το Google Drive μας, που περιέχει τα δεδομένα που απαιτούνται για το project. Χρησιμοποιούμε ενα relative path για την πρόσβαση στα δεδομένα "/content/drive/MyDrive/data/..."

Στο παρακάτω κελί, ο κώδικας συγκεντρώνει τα δεδομένα στα ζητούμενα αρχεία train_2019/2023.csv

**Παρατήρηση:** Παίρνουμε τη στήλη neighbourhood_cleansed, διότι οι γειτονίες είναι πιο καθαρογραμμένες και κάνει την ομαδοποίηση των δεδομένων πιο εύκολη.

In [ ]:
import pandas as pd
import sys

wantedColumns = ['id', 'zipcode', 'transit', 'bedrooms', 'beds',
                'review_scores_rating', 'number_of_reviews',
                'neighbourhood_cleansed', 'name', 'latitude', 'longitude',
                'last_review', 'instant_bookable', 'host_since',
                'host_response_rate', 'host_identity_verified',
                'host_has_profile_pic', 'first_review', 'description',
                'city', 'cancellation_policy', 'bed_type', 'bathrooms',
                'accommodates', 'amenities', 'room_type', 'property_type',
                'price', 'availability_365', 'minimum_nights', 'host_id']

data = {
    '2019': ['febrouary', 'march', 'april'],
    '2023': ['march', 'june', 'september']
}
data_2019 = []
data_2023 = []

for year, months in data.items():
  for month in months:
    file_path = f'/content/drive/MyDrive/data/{year}/{month}/listings.csv'
    df = pd.read_csv(file_path)
    df = df.filter(wantedColumns)
    df = df.rename(columns={'neighbourhood_cleansed':'neighbourhood'})
    df['Month'] = month
    if year == '2019':
      data_2019.append(df)
    else:
      data_2023.append(df)

combined_data_2019 = pd.concat(data_2019)
combined_data_2023 = pd.concat(data_2023)
combined_data_2023
combined_data_2019.to_csv('/content/drive/MyDrive/data/2019/train_2019.csv')
combined_data_2023.to_csv('/content/drive/MyDrive/data/2023/train_2023.csv')

* Τώρα φιλτράρουμε τις τιμές NaN συμπληρώνοντας τις αριθμητικές στήλες με τη μέση τιμή της στήλης, σημειώνοντας τις εξαιρέσεις όπου ολόκληρες στήλες είναι κενές. Για τις στήλες συμβολοσειρών, τις αντικαθιστούμε με άγνωστο.

* Για υπερβολικές τιμές, αναλύσαμε κάθε στήλη αποφασίσαμε ότι:

  Οι τιμές για τις περισσότερες αριθμητικές στήλες ήταν ρεαλιστικές και επομένως δεν χρειάζονταν επιπλέον φιλτράρισμα.

In [ ]:
import pandas as pd
import numpy as np

dfs = {}  # USE AN ARRAY TO STORE THE DFS OF EACH YEAR SEPARATELY FOR FUTURE PROCESSING
years = ['2019','2023']
for year in years:
  file_path = f'/content/drive/MyDrive/data/{year}/train_{year}.csv'
  df = pd.read_csv(file_path)

  for col in df.select_dtypes(include=np.number).columns:
    if df[col].isna().all(): # CHECK IF THE ENTIRE COLUMN IS NAN
      df[col].fillna(0, inplace=True)  # FILL WITH 0 WHERE THE ENTIRE COLUMN IS NAN (2023 bathrooms)
    else:
      df[col].fillna(df[col].mean(), inplace=True) # FILL NAN VALUES WITH THE MEAN NUMERICAL VALUE

  # FOR NON NUMERICAL COLUMNS, USE THE KEYWORD 'UNKNOWN' FOR NAN VALUES
  for col in df.select_dtypes(exclude=np.number).columns:
    df[col].fillna('Unknown', inplace=True)

  # DROP NON-SALVAGABLE NAN VALUES
  df.dropna(inplace=True)

  dfs[year] = df  # STORE THE DATAFRAMES ACCORDINGLY

Διαχωρίσαμε τα δεδομένα σε df_2019 και df_2023 για μελλοντική χρήση. Για ορισμένα ερωτήματα, ορίσαμε το df_unique_XXXX το οποίο παίρνει την πιο πρόσφατη εμφάνιση (κατά μήνα) με μοναδικό αναγνωριστικό, απορρίπτοντας όλα τα άλλα αντίγραφα. Επομένως, δεν έχουμε επαναλαμβανόμενες καταχωρήσεις εκεί που δεν χρειάζονται. Χρησιμοποιούμε το αντίστοιχο σύνολο δεδομένων ανάλογα με τις περιστάσεις.

In [ ]:
df_2019 = dfs['2019']
df_2023 = dfs['2023'] # WE DEFINE EACH DATASET FOR FUTURE USE WITHIN THE QUERIES

df_unique_2019 = df_2019[df_2019['Month'] == 'april'].drop_duplicates(subset=['id'])
df_unique_2023 = df_2023[df_2023['Month'] == 'september'].drop_duplicates(subset=['id']) # DENOTE UNIQUE VERSIONS OF THE DATASETS

**1.1** Ποιός είναι ο πιο συχνός τύπος room_type για τα δεδομένα σας;

In [ ]:
most_frequent_room_type_2019 = df_unique_2019['room_type'].mode()[0]
print(f"The most frequent room_type for 2019 is: {most_frequent_room_type_2019}")

most_frequent_room_type_2023 = df_unique_2023['room_type'].mode()[0]
print(f"The most frequent room_type for 2023 is: {most_frequent_room_type_2023}")

**1.2** Φτιάξτε γράφημα ή γραφήματα που δείχνουν την πορεία των τιμών για το διάστημα των 3 μηνών.  
* Αποφασίσαμε να δημιουργήσουμε δύο διαφορετικά γραφήματα για να αποφύγουμε προβλήματα με τις τιμές NaN και τη λογική απεικόνισης. Χρησιμοποιούμε επίσης το μέσο όρο της τιμής για κάθε μήνα για τα γραφήματα.
*Για αυτό το ερώτημα, χρειαζόμαστε μηνιαία δεδομένα, επομένως δεν χρησιμοποιούμε τα μοναδικά σύνολα.

In [ ]:
import matplotlib.pyplot as plt

# WE REMOVE THE $ SYMBOL SO WE CAN TAKE THE VALUES AS NUMERALS (float)
df_2019['price'] = df_2019['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df_2023['price'] = df_2023['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# WE GROUP BY MONTH AND GET THE MEAN OF THE PRICES (FOR EACH MONTH ENTRY)
df_2019_grouped = df_2019.groupby('Month')['price'].mean()
df_2023_grouped = df_2023.groupby('Month')['price'].mean()

# REORDER THE INDEX FOR EACH YEAR
df_2019_grouped = df_2019_grouped.reindex(['febrouary', 'march', 'april'])
df_2023_grouped = df_2023_grouped.reindex(['march', 'june', 'september'])

# PLOT CREATIONS

# 2019
plt.figure(figsize=(10, 6))
df_2019_grouped.plot(label='2019')
plt.title('Price Trend Over Specific Months in 2019')
plt.xlabel('Month')
plt.ylabel('Average Price')
plt.legend()
plt.xticks(range(len(df_2019_grouped)), df_2019_grouped.index)
plt.show()

print("\n")
# 2023
plt.figure(figsize=(10, 6))
df_2023_grouped.plot(label='2023')
plt.title('Price Trend Over Specific Months in 2023')
plt.xlabel('Month')
plt.ylabel('Average Price')
plt.legend()
plt.xticks(range(len(df_2023_grouped)), df_2023_grouped.index)
plt.show()

**1.3** Ποιές είναι οι 5 πρώτες γειτονιές με τις περισσότερες κριτικές;
* Εδώ εκπληρώσαμε την απαιτούμενη λειτουργία, αλλά προσθέσαμε επίσης έναν έλεγχο για την αφαίρεση του "Unknown", το οποίο τοποθετήσαμε για να συμπληρώσουμε τιμές NaN κατά την επεξεργασία του συνόλου δεδομένων μας (2023). Εν ολίγοις, δημιουργήσαμε μια άλλη έκδοση του συνόλου δεδομένων του 2023, η οποία αγνοεί τις εγγραφές με άγνωστη τοποθεσία.

In [ ]:
df_unique_2019_noU = df_unique_2019[df_unique_2019['neighbourhood'] != 'Unknown'] # EDIT THE SET TO FILTER OUR UNKNOWN NEIGHBORHOODS
df_unique_2023_noU = df_unique_2023[df_unique_2023['neighbourhood'] != 'Unknown'] # EDIT THE SET TO FILTER OUR UNKNOWN NEIGHBORHOODS

top_5_neighborhoods_2019 = df_unique_2019_noU.groupby('neighbourhood')['number_of_reviews'].sum().nlargest(5) # GET THE 5 LARGEST SUM OF REVIEWS FOR THE SPECIFIED YEAR
print("Top 5 neighborhoods with the most reviews in 2019:")
print(top_5_neighborhoods_2019)

# REPEAT FOR 2023
top_5_neighborhoods_2023 = df_unique_2023_noU.groupby('neighbourhood')['number_of_reviews'].sum().nlargest(5)
print("\nTop 5 neighborhoods with the most reviews in 2023:")
print(top_5_neighborhoods_2023)

**1.4** Ποιά είναι η γειτονιά με τις περισσότερες καταχωρήσεις ακινήτων;

In [ ]:
top_neighborhood_2019 = df_unique_2019['neighbourhood'].value_counts().idxmax()
top_neighborhood_2023 = df_unique_2023['neighbourhood'].value_counts().idxmax()

print(f"The neighborhood with the most unique property listings in 2019 is: {top_neighborhood_2019}")
print(f"The neighborhood with the most unique property listings in 2023 is: {top_neighborhood_2023}")

**1.5** Πόσες είναι οι καταχωρήσεις ανά γειτονιά και ανά μήνα;

**Παρατήρηση:** Με βάση συζήτησης στο e-class αποφασίσαμε να κάνουμε το **1.5** σε μορφή ιστογράμματος για να καλύψουμε και το ερώτημα **1.6**.

In [ ]:
# GROUP THE DATAFRAMES BY 'NEIGHBOURHOOD' AND 'MONTH', COUNT THE NUMBER OF ENTRIES FOR EACH GROUP, AND RESHAPE THE RESULT INTO A DATAFRAME
entries_per_neighborhood_month_2019 = df_2019.groupby(['neighbourhood', 'Month']).size().unstack()
entries_per_neighborhood_month_2023 = df_2023.groupby(['neighbourhood', 'Month']).size().unstack()

# PLOT A HORIZONTAL BAR CHART FOR THE 2019 DATA
entries_per_neighborhood_month_2019.plot(kind='barh', figsize=(13,15), edgecolor='white', linewidth=0.5)
plt.title('Number of entries per neighborhood and per month in 2019')
plt.ylabel('Number of entries')
plt.grid(axis='x', linestyle='-', alpha=0.3)
plt.show()

print("\n\n")

# PLOT A HORIZONTAL BAR CHART FOR THE 2023 DATA
entries_per_neighborhood_month_2023.plot(kind='barh', figsize=(13,15), edgecolor='white', linewidth=0.5)
plt.title('Number of entries per neighborhood and per month in 2023')
plt.ylabel('Number of entries')
plt.grid(axis='x', linestyle='-', alpha=0.3)
plt.show()

**1.6** Σχεδιάστε το ιστόγραμμα της μεταβλητής neighborhood
* Για αυτό το ερώτημα, χρησιμοποιήσαμε τα μοναδικά σύνολα δεδομένων και επιλέξαμε να αποκλείσουμε τις καταχωρήσεις στο σύνολο 2023 με λιγότερες από 3 εμφανίσεις λόγω των πολλών μοναδικών ονομάτων γειτονιάς εντός του σύνολου.
* **Με βάση τις συζητήσεις στο eclass, αυτό το ερώτημα δεν χρειάζεται να υλοποιηθεί. Εφόσον το έχουμε ήδη υλοποιήσει, απλά το βαλαμε σε σχόλιο.**

In [ ]:
# neighborhood_counts_2019 = df_unique_2019['neighbourhood'].value_counts() # WE USE THE UNIQUE DATASET

# # PLOT THE HISTOGRAM FOR 2019
# plt.figure(figsize=(10,6))
# neighborhood_counts_2019.plot(kind='bar')
# plt.title('Histogram of Neighborhood')
# plt.xlabel('Neighbourhood')
# plt.ylabel('Count')
# plt.show()

# # WE CHOSE TO EXCLUDE VALUES WITH LESS THAN 3 ENTRIES DUE TO THE FORMAT OF THE DATA
# neighborhood_counts_2023 = neighborhood_counts_2023[neighborhood_counts_2023 > 2]

# # 2023
# plt.figure(figsize=(10,6))
# neighborhood_counts_2023.plot(kind='bar')
# plt.title('Histogram of Neighborhood')
# plt.xlabel('Neighbourhood')
# plt.ylabel('Count')
# plt.show()


**1.7** Ποιος είναι ο πιο συχνός τύπος δωματίου (room_type) σε κάθε γειτονιά
(neighborhood);


In [ ]:
print("#############################################################\n\n 2019: \n\n############################################################# \n\n")
most_common_room_type_2019 = df_unique_2019.groupby('neighbourhood')['room_type'].agg(lambda x: x.value_counts().index[0])
print(most_common_room_type_2019)

print("\n\n#############################################################\n\n 2023: \n\n############################################################# \n\n")
most_common_room_type_2023 = df_unique_2023.groupby('neighbourhood')['room_type'].agg(lambda x: x.value_counts().index[0])
print(most_common_room_type_2023)

**1.8** Ποιός είναι ο πιο ακριβός τύπος δωματίου;
* Για αυτό το ερώτημα, δημιουργούμε νέα σύνολα δεδομένων για να έχουμε πιο ακριβή αποτελέσματα.

In [ ]:
# WE USE A NEW TEMPORARY DATASET AND DROP DUPLICATES THAT AREN'T THE HIGHEST PRICE (OF THE DUPLICATE ENTRIES)
df_2019_18 = df_2019.sort_values('price', ascending=False).drop_duplicates(subset=['id'])
df_2023_18 = df_2023.sort_values('price', ascending=False).drop_duplicates(subset=['id'])

# WE GROUP BY ROOM TYPE AND GET THE MEAN OF EACH TYPE
mean_price_2019 = df_2019_18.groupby('room_type')['price'].mean()
mean_price_2023 = df_2023_18.groupby('room_type')['price'].mean()

# WE SELECT THE HIGHEST MEAN OF THE ROOM TYPES.
most_expensive_room_type_2019 = mean_price_2019.idxmax()
most_expensive_room_type_2023 = mean_price_2023.idxmax()

print(f"The most expensive room type in 2019 is {most_expensive_room_type_2019}")
print(f"The most expensive room type in 2023 is {most_expensive_room_type_2023}")

**1.9** Χρησιμοποιήστε τη βιβλιοθήκη Folium Map με τις στήλες latitude/longitude και
εμφανίστε σε ένα χάρτη για ένα μήνα της επιλογής σας τα ακίνητα και στα popup στον
χάρτη επιλέξτε όποια άλλη πληροφορία θέλετε να εμφανίζεται για το ακίνητο (πχ
bed_type, room_type, transit κτλ).
* Για αυτό το ερώτημα, έπρεπε να χρησιμοποιήσουμε ένα plugin του folium (MarkerCluster), καθώς η εκτέλεση των χαρτών έσπαγε τον ιστότοπο λόγω του πλήθους των ακινήτων που προσθέτονταν στο χάρτη.

In [ ]:
import folium
from folium.plugins import MarkerCluster

# 2019
m = folium.Map(location=[df_unique_2019['latitude'].mean(), df_unique_2019['longitude'].mean()], zoom_start=13) # GET A CENTER POINT BASED ON ALL THE LISTINGS

marker_cluster = MarkerCluster().add_to(m)

# LOOP TO ADD EACH PROPERTY TO THE MAP BASED ON THE PROVIDED LAT/LONG
for idx, row in df_unique_2019.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Room type: {row['room_type']}" # WE CHOSE TO DISPLAY THE ROOM_TYPE
    ).add_to(marker_cluster)

# DISPLAY THE FIRST MAP
print('##############################################################\n\n\n 2019 \n\n\n############################################################## \n')
display(m)

# 2023
m2 = folium.Map(location=[df_unique_2023['latitude'].mean(), df_unique_2023['longitude'].mean()], zoom_start=13) # GET A CENTER POINT BASED ON ALL THE LISTINGS

marker_cluster2 = MarkerCluster().add_to(m2)

# LOOP TO ADD EACH PROPERTY TO THE MAP BASED ON THE PROVIDED LAT/LONG
for idx, row in df_unique_2023.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Room type: {row['room_type']}" # WE CHOSE TO DISPLAY THE ROOM_TYPE
    ).add_to(marker_cluster2)

# 2023 MAP
print('\n##############################################################\n\n\n 2023 \n\n\n############################################################## \n')
display(m2)

**1.10** Φτιάξτε διαφορετικά wordclouds με τα δεδομένα από τη στήλη neighbourhood,
transit, description, last_review.
* Για αυτό το ερώτημα, παραλείψαμε την κατηγορία διαμετακόμισης για το σύνολο δεδομένων του 2023, καθώς δεν υπάρχει.
* Επίσης, για κάθε accomodation παίρνουμε μόνο την πιο πρόσφατη κριτική που υποβλήθηκε, για να ελαχιστοποιήσουμε τον όγκο των δεδομένων. (e-class)

In [ ]:
from wordcloud import WordCloud
# FILE PATHS FOR THE MOST RECENT FILE FOR EACH YEAR
file_path_2019_april = '/content/drive/MyDrive/data/2019/april/reviews.csv'
file_path_2023_september = '/content/drive/MyDrive/data/2023/september/reviews.csv'

dfN_2019_april = pd.read_csv(file_path_2019_april)
dfN_2019_april['date'] = pd.to_datetime(dfN_2019_april['date'])  # CONVERT THE 'DATE' COLUMN TO DATETIME
dfN_2019_april = dfN_2019_april.sort_values(['listing_id', 'date'])  # SORT THE DATAFRAME BY 'LISTINGID' AND 'DATE'
dfN_2019_april = dfN_2019_april.drop_duplicates('listing_id', keep='last')  # DROP DUPLICATES, KEEPING ONLY THE LAST COMMENT FOR EACH 'LISTINGID'

# READ AND PROCESS THE DATA FOR SEPTEMBER 2023
dfN_2023_september = pd.read_csv(file_path_2023_september)
dfN_2023_september['date'] = pd.to_datetime(dfN_2023_september['date'])  # CONVERT THE 'DATE' COLUMN TO DATETIME
dfN_2023_september = dfN_2023_september.sort_values(['listing_id', 'date'])  # SORT THE DATAFRAME BY 'LISTINGID' AND 'DATE'
dfN_2023_september = dfN_2023_september.drop_duplicates('listing_id', keep='last')  # DROP DUPLICATES, KEEPING ONLY THE LAST COMMENT FOR EACH 'LISTINGID'

text_2019 = dfN_2019_april['comments'].dropna().astype(str).str.cat(sep=' ')
text_2023 = dfN_2023_september['comments'].dropna().astype(str).str.cat(sep=' ')

# -------------------------

# DEFINE THE DATA FOR EACH YEAR
data = {'2019': df_unique_2019_noU, '2023': df_unique_2023_noU}

# DEFINE THE COLUMNS YOU'RE INTERESTED IN
columns = ['neighbourhood', 'description']

# ITERATE OVER THE YEARS
for year, df in data.items():
    # ITERATE OVER THE COLUMNS
    for column in columns:
        # CONCATENATE ALL VALUES IN THE COLUMN INTO A SINGLE STRING
        text = df[column].dropna().astype(str).str.cat(sep=' ')

        # GENERATE A WORD CLOUD
        wordcloud = WordCloud(background_color='white').generate(text)

        # PLOT THE WORD CLOUD
        plt.figure(figsize=(15, 15))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Word Cloud for {column} - {year}')
        plt.show()
        print("\n\n")

# GENERATE A WORD CLOUD FOR THE 'TRANSIT' COLUMN IN THE 2019 DATAFRAME
text = df_unique_2019['transit'].dropna().astype(str).str.cat(sep=' ')
wordcloud = WordCloud(background_color='white').generate(text)

# PLOT THE WORD CLOUD FOR THE 'TRANSIT' COLUMN
plt.figure(figsize=(15, 15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud for Transit - 2019')
plt.show()
print("\n\n")

# DEFINE THE DATA FOR EACH YEAR
dataN = {'2019': dfN_2019_april, '2023': dfN_2023_september}

# ITERATE OVER THE YEARS
for year, df in dataN.items():
    # CONCATENATE ALL VALUES IN THE 'COMMENTS' COLUMN INTO A SINGLE STRING
    text = df['comments'].dropna().astype(str).str.cat(sep=' ')

    # GENERATE A WORD CLOUD
    wordcloud = WordCloud(background_color='white').generate(text)

    # PLOT THE WORD CLOUD
    plt.figure(figsize=(15, 15))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Word Cloud for Comments - {year}')
    plt.show()
    print("\n\n")

**1.11** Στο ερώτημα αυτό θα αξιοποιήσουμε τη στήλη amenities. Στην συνέχεια
και αφού έχετε απλοποιήσει αυτήν τη στήλη κάντε ένα ιστόγραμμα το οποίο θα
κατανέμονται οι νέες τιμές.
* Για αυτό το ερώτημα, παίρνουμε τη στήλη amenities και χωρίζουμε τα strings σε ξεχωριστα amenities. Στη συνέχεια χρησιμοποιήσαμε κώδικα για να εκτυπώσουμε μη διπλές καταχωρήσεις στη στήλη amenities και τις κατηγοριοποιήσαμε στις παρεχόμενες κατηγορίες (προσθέτοντας δύο δικές μας). Φιλτράραμε επίσης τα περιττά κενά για καλύτερη οργάνωση και αποφυγή σφαλμάτων. Τέλος, ισοπεδώσαμε τις κατηγορίες για να τις χρησιμοποιήσουμε για τη γραφική παράσταση και σχεδιάσαμε το ζητούμενο γράφημα.


In [ ]:
# DEFINE THE AMENITIES INTO THE SPECIFIED CATEGORIES
categories = {
    'kitchen': ['Kitchen','Breakfast', 'Cooking basics', 'BBQ grill', 'Oven', 'Coffee maker','Dishes and silverware','Dishwasher','Refrigerator','Microwave', 'Gas oven', 'Children’s dinnerware', 'Kitchenette', 'Double oven', 'Full kitchen', 'Espresso machine', 'Steam oven', 'Hot water kettle', 'Convection oven', 'Mini fridge', 'Wine cooler', 'Breakfast table', 'Mudroom', 'Warming drawer'],
    'bathroom': ['Hot water', 'Hair dryer','Essentials','Shampoo','Bathtub', 'Accessible-height toilet', 'Private bathroom', 'Bath towel', 'Bathtub with bath chair', 'En suite bathroom', 'Toilet paper', 'Bathroom essentials', 'Rain shower', 'Roll-in shower', 'Jetted tub', 'Walk-in shower', 'Heat lamps', 'Bidet', 'Body soap', 'Alfresco bathtub', 'Soaking tub', ' toilet', 'Shower chair', 'Stand alone steam shower'],
    'accesibility': ['Wide doorway','Well-lit path to entrance','Step-free access','Elevator','Wide hallway clearance','Single level home', 'Flat path to front door', 'Ground floor access', 'Wide clearance to bed', 'Wide clearance to shower', 'Wide entryway', 'Disabled parking spot', 'Wheelchair accessible', 'Accessible-height bed', 'Fixed grab bars for toilet', 'Fixed grab bars for shower', 'Electric profiling bed'],
    'Electricity_and_Technology': ['TV', 'Cable TV', 'Internet', 'Wifi','Air conditioning', 'Keypad', 'Amazon Echo', 'Smart lock', 'Game console', 'High-resolution computer monitor', 'Printer', 'Projector and screen', 'Smart TV', 'HBO GO', 'Netflix', 'DVD player', 'Central air conditioning', 'Ethernet connection', 'Pocket wifi', 'EV charger'],
    'facilities': ['Washer', 'Dryer', 'Elevator','Iron','Stove','Patio or balcony','Heating','Private entrance','Room-darkening shades','Garden or backyard','Free parking on premises','Indoor fireplace', 'Freezer', 'Beachfront', 'Sun loungers', 'Fire pit', 'Private hot tub', 'Sauna', 'Shared gym', 'Terrace', 'Hot tub', 'Pool', 'Lake access', 'Private pool', 'Balcony', 'Ski-in/Ski-out', 'Ceiling fan', 'Outdoor parking', 'Heated floors', 'Heated towel rack', 'Touchless faucets', 'Standing valet', 'Outdoor seating', 'Waterfront', 'Washer / Dryer', 'Fireplace guards', 'Beach view'],
    'kids_friendly': ['Family/kid friendly','Crib', 'Babysitter recommendations', 'Children’s books and toys', 'Changing table', 'High chair', 'Table corner guards', 'Outlet covers', 'Baby monitor', 'Pack ’n Play/travel crib', 'Baby bath', 'Window guards', 'Murphy bed', 'Stair gates','Gym'],
    'security': ['Fire extinguisher','First aid kit','Carbon monoxide detector','Smoke detector','Buzzer/wireless intercom','Lockbox', 'Lock on bedroom door', 'Safety card'],
    'services': ['Long term stays allowed','Laptop friendly workspace','Extra pillows and blankets','Luggage dropoff allowed','24-hour check-in','Paid parking off premises','Host greets you','Free street parking','Self check-in', 'Cleaning before checkout', 'Building staff', 'Doorman', 'Fax machine'],
    'miscellaneous':['Hangers','EssentialsBed linens','Smoking allowed', 'Pets live on this property', 'Sound system', 'Firm mattress', 'Paid parking on premises', 'Other pet(s)', 'Pets allowed', 'Air purifier', 'Bedroom comforts', 'Memory foam mattress', 'Suitable for events', 'Ironing Board', 'Exercise equipment', 'Handheld shower head', 'Hammock', 'Day bed', 'Cat(s)', 'Dog(s)', 'Private living room', 'Other', 'Mountain view', 'Formal dining area', 'Pillow-top mattress', 'Beach essentials']
}

#REDIFINE THE UNIQUE DATASETS FOR EASE OF USE
df_unique_2019 = df_2019[df_2019['Month'] == 'april'].drop_duplicates(subset=['id'])
df_unique_2023 = df_2023[df_2023['Month'] == 'september'].drop_duplicates(subset=['id'])

# SPLIT THE STRING BY SEPARATING IT WHEN A , IS FOUND
df_unique_2019['amenities'] = df_unique_2019['amenities'].str.split(',')

# REMOVE LEADING AND TRAILING SPACES AND FILTER BASED ON THE 9 MAIN CATEGORIES
df_unique_2019['categories'] = df_unique_2019['amenities'].apply(lambda amenities: [category for category, items in categories.items() if any(item.strip() in amenities for item in items)])

# FLATTEN THE LIST OF CATEGORIES INTO A SINGLE SERIES USING EXPLODE
categories_series = df_unique_2019['categories'].explode()

# PLOTTING THE HISTOGRAM
categories_series.value_counts().plot(kind='bar')
plt.title('HISTOGRAM OF AMENITY CATEGORIES 2019')
plt.xlabel('CATEGORY')
plt.ylabel('FREQUENCY')
plt.show()

#2023
print("\n\n")
import ast

# 2023
print('##############################################################\n\n\n 2023 \n\n\n############################################################## \n')
df_unique_2023['amenities'] = df_unique_2023['amenities'].apply(ast.literal_eval) # WE USE AST TO CONVERT THE STRING ENCLOSED IN [] INTO AN ACTUAL LIST, AS WE CANNOT ITERATE IT OTHERWISE.
df_unique_2023['categories'] = df_unique_2023['amenities'].apply(lambda amenities: [category for category, items in categories.items() if any(item.strip() in [amenity.strip() for amenity in amenities] for item in items)])
categories_series2 = df_unique_2023['categories'].explode()

# CHECK IF ITS EMPTY BEFORE PLOTTING TO AVOID ERRORS

categories_series2.value_counts().plot(kind='bar')
plt.title('HISTOGRAM OF AMENITY CATEGORIES 2023')
plt.xlabel('CATEGORY')
plt.ylabel('FREQUENCY')
plt.show()

**1.12** Υπολογίστε αρχικά την μέση τιμή ανα γειτονιά (θα χρησιμοποιηθεί η στήλη price,
ενώ για να μπορούμε να συγκρίνουμε όμοια πράγματα κρατήστε μόνο τα δωμάτια που
επιτρέπουν φιλοξενία δύο ατόμων). Δείξτε σε ένα γράφημα τις γειτονιές ταξινομημένες
ανάλογα με την μέση τιμή τους. Στην συνέχεια κατατάξτε τις γειτονιές σε 3 ομάδες (πολύ
ακριβές, μέτριες, οικονομικές) ανάλογα με την μέση τιμή των δωματίων.
* Για αυτό το ερώτημα, υπολογίζουμε τον μέσο όρο για κάθε γειτονιά χωρίς διπλές εγγραφές. Στη συνέχεια ορίζουμε τις ζητούμενες κατηγορίες και συγχωνεύουμε τις απαιτούμενες στήλες, έπειτα τις ταξινομούμε και τέλος εκτυπώνουμε το γράφημα με κωδικοποίηση χρώματος.

*   Για το έτος 2023, έχουμε δεδομένα από 2 λιγότερες γειτονιές αυτό οφείλεται στο γεγονός πως εκείνο το έτος και μήνα δεν υπήρχε κάποιο airbnb για 2 άτομα.


In [ ]:
import matplotlib.pyplot as plt

# 2019
# CALCULATE AVG PRICES PER NEIGHBOURHOOD
df_two_persons = df_unique_2019[df_unique_2019['accommodates'] == 2].copy() # MAKE NEW DF COPY THAT ONLY CONTAINS ROOMS WITH 2 ACCOMMODATES
df_two_persons['price'] = df_two_persons['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)  # REMOVE THE $ SYMBOL AND TURN THEM TO FLOAT
average_price = df_two_persons.groupby('neighbourhood')['price'].mean().rename('average_price') # CALCULATE THE AVERAGE PRICE

# DEFINE PRICE CATEGORIES
bins = [0, average_price.quantile(0.33), average_price.quantile(0.66), average_price.max()] # PRICE BOUNDARIES BASED ON THE QUANTILES OF THE AVG PRICE
labels = ['Economical', 'Moderate', 'Expensive']  # THE PRICE CATEGORIES
neighbourhood_category = pd.cut(average_price, bins=bins, labels=labels)  # CREATE A NEW COLUMN 'NEIGHBOURHOOD_CATEGORY'

# MERGE NEIGHBRHOODS, AVG PRICES & CATEGORIES
neighbourhood_prices = pd.DataFrame(average_price).merge(neighbourhood_category.rename('neighbourhood_category'), left_index=True, right_index=True)
neighbourhood_prices = neighbourhood_prices.sort_values(by='average_price') # SORT BY THE AVERAGE_PRICE
# display(neighbourhood_prices)
print("Number of rows in neighbourhood_prices: ", len(neighbourhood_prices))
color_map = {'Economical': 'green', 'Moderate': 'yellow', 'Expensive': 'red'} # CREATE A COLOR MAP FOR THE PRICE CATEGORIES

# PLOT THE GRAPH
plt.figure(figsize=(13,8))
for category in labels:
    plt.bar(neighbourhood_prices[neighbourhood_prices['neighbourhood_category'] == category].index, # FET ALL NEIGHBOURHOODS OF THE CATEGORY
            neighbourhood_prices[neighbourhood_prices['neighbourhood_category'] == category]['average_price'], # AVERAGE PRICES
            color=color_map[category])  # COLOR BASED ON CATEGORY
plt.xticks(rotation=90)
plt.show()
print("\n")

# 2023
# CALCULATE AVG PRICES PER NEIGHBOURHOOD
df_two_persons = df_unique_2023[df_unique_2023['accommodates'] == 2].copy() # MAKE NEW DF COPY THAT ONLY CONTAINS ROOMS WITH 2 ACCOMMODATES
df_two_persons['price'] = df_two_persons['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)  # REMOVE THE $ SYMBOL AND TURN THEM TO FLOAT
average_price = df_two_persons.groupby('neighbourhood')['price'].mean().rename('average_price') # CALCULATE THE AVERAGE PRICE

# DEFINE PRICE CATEGORIES
bins = [0, average_price.quantile(0.33), average_price.quantile(0.66), average_price.max()] # PRICE BOUNDARIES BASED ON THE QUANTILES OF THE AVG PRICE
labels = ['Economical', 'Moderate', 'Expensive']  # THE PRICE CATEGORIES
neighbourhood_category = pd.cut(average_price, bins=bins, labels=labels)  # CREATE A NEW COLUMN 'NEIGHBOURHOOD_CATEGORY'

# MERGE NEIGHBRHOODS, AVG PRICES & CATEGORIES
neighbourhood_prices = pd.DataFrame(average_price).merge(neighbourhood_category.rename('neighbourhood_category'), left_index=True, right_index=True)
neighbourhood_prices = neighbourhood_prices.sort_values(by='average_price') # SORT BY THE AVERAGE_PRICE

color_map = {'Economical': 'green', 'Moderate': 'yellow', 'Expensive': 'red'} # CREATE A COLOR MAP FOR THE PRICE CATEGORIES

# display(neighbourhood_prices)
print("Number of rows in neighbourhood_prices: ", len(neighbourhood_prices))
# PLOT THE GRAPH
plt.figure(figsize=(13,8))
for category in labels:
    plt.bar(neighbourhood_prices[neighbourhood_prices['neighbourhood_category'] == category].index, # GET ALL NEIGHBOURHOODS OF THE CATEGORY
            neighbourhood_prices[neighbourhood_prices['neighbourhood_category'] == category]['average_price'], # AVERAGE PRICES
            color=color_map[category])  # COLOR BASED ON CATEGORY
plt.xticks(rotation=90)
plt.show()

**1.13** Ποιά άλλη πληροφορία θα μπορούσατε να εξάγετε από τα δεδομένα που σας
δίνονται; Σκεφτείτε 3 επιπλέον ερωτήσεις για την περιοχή της Αθήνας και εμφανίστε τα
αποτελέσματα (μπορείτε και να συνδυάσετε όσες στήλες θέλετε από οποιαδήποτε
χρονιά).
* Ποιο είναι το λιγότερο διαθέσιμο "room_type" με βάση την 365_availability;
* Ποιος είναι ο πιο συνηθισμένος ελάχιστος αριθμός διανυκτερεύσεων που απαιτείται για καταχωρήσεις στην Αθήνα;
* Ποιος είναι ο μέσος αριθμός ατόμων που μπορεί να φιλοξενήσει μια καταχώριση στην Αθήνα;

In [ ]:
# 2019
# 1
least_available_room_type_2019 = df_unique_2019.groupby('room_type')['availability_365'].mean().idxmin()
print(f"The least available room type in 2019 is: {least_available_room_type_2019}")

# 2
most_common_minimum_nights_2019 = df_unique_2019['minimum_nights'].mode().values[0]
print(f"The most common minimum number of nights in 2019 is: {most_common_minimum_nights_2019}")

# 3
average_accommodates_2019 = df_unique_2019['accommodates'].mean()
print(f"The average number of people a listing can accommodate in 2019 is: {average_accommodates_2019:.2f}")

print("\n")

# 2023
# 1
least_available_room_type_2023 = df_unique_2023.groupby('room_type')['availability_365'].mean().idxmin()
print(f"The least available room type in 2023 is: {least_available_room_type_2023}")

# 2
most_common_minimum_nights_2023 = df_unique_2023['minimum_nights'].mode().values[0]
print(f"The most common minimum number of nights in 2023 is: {most_common_minimum_nights_2023}")

# 3
average_accommodates_2023 = df_unique_2023['accommodates'].mean()
print(f"The average number of people a listing can accommodate in 2023 is: {average_accommodates_2023:.2f}")

**1.14**

In [ ]:
#REDEFINE THE UNIQUE DATASETS
df_unique_2019 = df_2019[df_2019['Month'] == 'april'].drop_duplicates(subset=['id'])
df_unique_2023 = df_2023[df_2023['Month'] == 'september'].drop_duplicates(subset=['id'])

#2019
# SUM THE LISTINGS ON THE UNIQUE DATASET BY EACH HOST ID
host_listings = df_unique_2019.groupby('host_id').size()

# USE HEAD(10) TO LIST THE TOP 10
top_10_hosts = host_listings.sort_values(ascending=False).head(10)

print("Top 10 hosts with the most accomodations for 2019\n")
print(top_10_hosts)
print("\n")

#2023
# SUM THE LISTINGS ON THE UNIQUE DATASET BY EACH HOST ID
host_listings = df_unique_2023.groupby('host_id').size()

# USE HEAD(10) TO LIST THE TOP 10
top_10_hosts = host_listings.sort_values(ascending=False).head(10)

print("Top 10 hosts with the most accomodations for 2023\n")
print(top_10_hosts)

**1.15**

Από τα παραπάνω ερωτήματα μπορούμε να βγάλουμε διάφορα συμπεράσματα για τις βραχύχρονες μισθώσεις στην Αθήνα.

* Και τις δύο χρονίες παρατηρούμε από το γράφημα του ερωτήματος **1.2** πως οι τιμές των καταλυμάτων είναι χαμηλότερες την εποχή της άνοιξης, κάτι που είναι πολύ φυσιολογικό αφού η εποχή αυτή θεωρείται εκτός σεζόν.

* Από τα ερωτήματα 1.3 - 1.6 & 1.9 παρατηρούμε πως οι πιο *ενεργές* περιοχές είναι αυτές που βρίσκονται κοντά στο κέντρο. Αυτό το καταλαβαίνουμε από το μεγάλο πλήθος κριτικών που έχουν τα καταλύματα σε αυτές τις περιοχές καθώς και από το μεγάλο αριθμό καταχωρημένων airbnb που θα βρούμε σε αυτές τις περιοχές όπως παρατηρούμε από τις κατανομημένες λίστες που έχουμε φτιάξει (1.3), τα ιστογραφήματα (1.5) αλλά ακόμα και από τη πυκνότητα που έχουν οι χάρτες μας σε αυτές τις περιοχές (1.9). Τέλος να σημειωθεί πως η πιο δημοφιλή από αυτές τις περιοχές, η "ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ", από το 2019 στο 2023 κατάφερε να μεγαλώσει τη διαφορά που έχει στις καταχωρήσεις συγκριτικά με τις άλλες περιοχές, άρα καταλήγουμε στο συμπέρασμα πως η τάση των airbnb να αυξάνονται προς το κέντρο της πόλης δεν φαίνεται να σταματάει.

* Ένα κοινό χαρακτηριστικό που έχουν και οι δύο χρονιές σε όλες τις περιοχές είναι πως νοικιάζονται κυρίως καταλύματα που προσφέρουν ολόκληρο διαμέρισμα και όχι απλά ένα δωμάτιο σε κάποιο ξενοδοχείο ή σπίτι (1.7). Αυτό εξηγεί κιόλας γιατί πολλά amenities έχουν να κάνουν με κατηγορίες όπως π.χ. η κουζίνα (1.11).

* Από το ερώτημα **1.12** παρατηρούμε πως το 2023 υπάρχει σημαντική αύξηση στις τιμές των καταλυμάτων σε όλες τις κατηγορίες τιμών. Οι περισσότερες ακριβές περιοχές, συνεχίζουν να είναι ακριβές αν και δεν είναι με την ίδια ακριβώς σειρά στη κατάταξη.

* Ο παρακάτω κώδικας απαντάει στην πρώτη ερώτηση που έχει τεθεί στην εκφώνηση.

In [ ]:
# REDEFINE THE UNIQUE DATASETS
df_unique_2019 = df_2019[df_2019['Month'] == 'april'].drop_duplicates(subset=['id'])
df_unique_2023 = df_2023[df_2023['Month'] == 'september'].drop_duplicates(subset=['id'])

# Q1: ARE THERE MORE/DIFFERENT NEIGHBORHOODS BETWEEN THE TWO YEARS?
print("Q1: Are there more/different neighborhoods between the two years?\n")

num_neighbourhoods_2019 = df_unique_2019['neighbourhood'].nunique() # FILTER DATAFRAME FOR EACH YEAR AND COUNT UNIQUE NEIGHBORHOODS
num_neighbourhoods_2023 = df_unique_2023['neighbourhood'].nunique()

print(f"Number of unique neighborhoods in 2019: {num_neighbourhoods_2019}")
print(f"Number of unique neighborhoods in 2023: {num_neighbourhoods_2023}")

is_same_num = num_neighbourhoods_2019 == num_neighbourhoods_2023  # CHECK IF THE NUMBER OF NEIGHBORHOODS IS THE SAME
print(f"Did the number of neighborhoods change from 2019 to 2023? {is_same_num}\n")

neighbourhoods_2019 = set(df_unique_2019['neighbourhood'])  # GET UNIQUE NEIGHBORHOODS FOR EACH YEAR
neighbourhoods_2023 = set(df_unique_2023['neighbourhood'])

are_same = neighbourhoods_2019 == neighbourhoods_2023 # CHECK IF THE NEIGHBORHOODS ARE THE SAME
print(f"Are the neighborhoods in 2019 the same as in 2023? {are_same}\n")

# **Ερώτημα 2ο: Recommendation System**
Σε αυτό το ερώτημα θα χρειαστείτε τις στήλες : 'id', 'name', 'description'.
Σκοπός είναι να εξάγετε χρήσιμη πληροφορία από αυτά τα δεδομένα και να
προσπαθήσετε να φτιάξετε ένα πρόγραμμα το οποίο θα παράγει προτάσεις
(recommendations) για την περιοχή της Αθήνας. Στο πρώτο ερώτημα έχετε ήδη φτιάξει
τα wordclouds για τη στήλη description. Σε αυτό το ερώτημα αφαιρέστε τα stop words,
πειραματιστείτε με τις παραμέτρους του wordcloud και εντοπίστε τις πιο
χαρακτηριστικές λέξεις που χρησιμοποιεί ο επισκέπτης για την περιοχή της Αθήνας.
Στη συνέχεια δημιουργήστε μία νέα στήλη που θα έχει την ένωση (concatenation) των
στηλών name και description (fill NA with NULL).

* [ECLASS] για αυτό το ερώτημα, πέρνουμε μόνο μία χρονία (2019)


**2.1** Δημιουργήστε τον TF-IDF (Term Frequency - Inverse Document Frequency)
πίνακα των unigrams και των bigrams για τη νέα στήλη (χρησιμοποιήστε την
παράμετρο stop_word του TfidfVectorizer).

<!-- Στο συγκεκριμένο ερώτημα αφαιρούμε τα διπλότυμα ids για να αποφύγουμε τη σύγκριση παρόμοιων περιγραφών και παίρνουμε μόνο ένα δείγμα των δεδομένων για τη καλύτερη διαχείριση των δεδομένων και τη γρηγορότερη επεξεργασία τους. -->

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# DROP DUPLICATES
df_unique_2019 = df_2019[df_2019['Month'] == 'april'].drop_duplicates(subset=['id'])

# ASSIGNMENT DATA HANDLING REQUESTS
df_unique_2019['name'].fillna('NULL', inplace=True)
df_unique_2019['description'].fillna('NULL', inplace=True)
df_unique_2019['name_description'] = df_unique_2019['name'] + ' ' + df_unique_2019['description']

# GET 5% OF THE DATA
# df_2019_sample = df_unique_2019.sample(frac=0.80)

# CREATE NEW VECTORIZER
vectorizer_2019 = TfidfVectorizer(stop_words='english', ngram_range=(1, 2)) # ENGLISH = KNOWN DATASET OF COMMON WORDS IN ENGLISH

# USE THE NEW COLUMN FOR THE VECTORIZER FOR 2019
tfidf_matrix_2019 = vectorizer_2019.fit_transform(df_unique_2019['name_description'])


**2.2** Cosine Similarity: Η μετρική αυτή υπολογίζει την ομοιότητα μεταξύ δύο
διανυσμάτων x,y, χρησιμοποιώντας τη γωνία μεταξύ τους (όταν η γωνία είναι 0
σημαίνει ότι τα x και y είναι ίσα , αν εξαιρέσουμε το μήκος τους). Διατρέξτε τον
TF-IDF πίνακα και υπολογίστε το similarity καθενός ακινήτου με τα υπόλοιπα.

Τις ομοιότητες κάθε ακινήτου με τα υπόλοιπα τις αποθηκεύουμε σε ένα dictionary όπου χρησιμοποιούμε ως κλειδί το id του ακινήτου αυτού. Τα ακίνητα αυτά είναι ταξινομημένα με βάση το σκορ ομοιότητας.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# CALCULATE COSINE SIMILARITY BETWEEN EACH ENTRY (CRASHES)
similarity_2019 = cosine_similarity(tfidf_matrix_2019)

# DICTIONARIES TO STORE THE TOP 100
most_similar_2019 = {}

# COMPARE THE RESULTS TO FIND THE TOP 100 FOR 2019
for i in range(len(similarity_2019)):
    similar_indices = np.argsort(similarity_2019[i])[::-1]
    key = df_unique_2019.iloc[i]['id']  # GET THE ID OF THE ITEM
    most_similar_2019[key] = df_unique_2019.iloc[similar_indices[1:101]]['id'].values  # EXCLUDE ITSELF FROM THE SEARCH

**2.3** Πρόβλεψη : Φτιάξτε μία συνάρτηση η οποία παίρνει σαν είσοδο ένα id και έναν
ακέραιο αριθμό N, και επιστρέφει τα Ν πιο όμοια ακίνητα.


*   Ελέγχουμε αν το ακίνητο υπάρχει στο δείγμα που έχουμε πάρει
*   Παίρνουμε τα ακίνητα με τη μεγαλύτερη ομοιότητα για το ακίνητο που κάνουμε τη πρόβλεψη
*   Εκτυπώνουμε τα αποτελέσματα (Το όνομα και τη περιγραφή τα βρίσκουμε με βάση το id και τη σκορ ομοιότητας με βάση το index)



In [ ]:
import random

def recommend(item_id, num): # FUNCTION DECLARATION num = N
    # CHECK IF THE item_id EXISTS IN THE DATAFRAME
    if item_id not in df_unique_2019['id'].values:
        print(f"No property found with ID {item_id}")
        return

    # GET THE INDICES OF THE HIGHEST COSINE SIMILARITY PROPERTIES
    similar_item_ids = most_similar_2019[item_id][:num]

    # PRINT THE RECOMMENDATIONS IN THE REQUIRED FORMAT
    print(f"Recommending {num} listings similar to \"{df_unique_2019[df_unique_2019['id'] == item_id]['name'].values[0]}\"")
    print("---------------------------------------------------------")
    for similar_item_id in similar_item_ids:
        print(f"Recommended: {df_unique_2019[df_unique_2019['id'] == similar_item_id]['name'].values[0]}")
        print(f"Description: {df_unique_2019[df_unique_2019['id'] == similar_item_id]['description'].values[0]}")
        print(f"(score: {similarity_2019[df_unique_2019['id'].values.tolist().index(item_id)][df_unique_2019['id'].values.tolist().index(similar_item_id)]})\n")

random_id = random.choice(df_unique_2019['id'].values)  # SELECT A RANDOM ITEM_ID
print(random_id)
recommend(random_id, 5)

**2.4** Λέξεις που εμφανίζονται συχνά μαζί με άλλες λέξεις (collocation).
Χρησιμοποιήστε τον BigramCollocationFinder για να βρείτε 10 words που
“τείνουν” να εμφανίζονται συχνά μαζί.

*   Τη πρώτη φορά ο χρόνος εκτέλεσης είναι παραπάνω λόγω της εγκατάστασης της βιβλιοθήκης nltk

In [ ]:
import nltk
nltk.download('punkt')
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

# CONCATENATE ALL DESCRIPTIONS INTO A SINGLE STRING
all_descriptions = ' '.join(df_unique_2019['description'])

# TOKENIZE THE STRING INTO WORDS
words = nltk.word_tokenize(all_descriptions)

# CREATE A BigramCollocationFinder AND APPLY THE FREQUENCY FILTER
bigram_finder = BigramCollocationFinder.from_words(words)
bigram_finder.apply_freq_filter(3)

# FIND THE TOP 10 BIGRAMS USING THE PMI MEASURE
top_10_bigrams = bigram_finder.nbest(BigramAssocMeasures.pmi, 10)

# PRINT THE TOP 10 BIGRAMS
for bigram in top_10_bigrams:
    print(' '.join(bigram))